# ERP SALES INVOICE DATA SAMPLE INTERACTIVE DASHBOARD

The purpose of this document is to demonstrate the interactive graphing capabilities of the Python programming language. The visualizations in this example are generated using a synthetic dataset which is stored in a secure online repository and shared via Jupyter notebooks. Find out more at https://jupyter.org/ 

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import calendar
import csv
import random

%matplotlib inline
sns.set()

pd.set_option('display.float_format', lambda x: '%.2f' % x)


from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
init_notebook_mode(connected=True)
import plotly.graph_objects as go

from mlxtend.frequent_patterns import apriori, association_rules

import requests
import io

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
username = 'azzaroonie'
token = '01a4af5ad6aa740f02fa44dab39bd72d9e250134'

github_session = requests.Session()
github_session_auth = (username, token)

url = 'https://raw.githubusercontent.com/azzaroonie/stashorama/master/synth_rev2.csv?token=APBK6OXCNFYERX32HVTMVP27YMYO4'
download = github_session.get(url).content

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [4]:
saleval = pd.pivot_table(df, index = ['year','pricing_structure'], values = ['cost_value','sale_value','margin'], aggfunc = {'cost_value':'sum','sale_value':'sum','margin':'sum'})
saleval = saleval.reset_index(drop = False)

In [5]:
fig = px.bar(saleval, x = 'year', y = 'sale_value', color = 'pricing_structure', title='2017 - 2019 Parts Sales by Customer Discount Structure')
fig.update_xaxes(type='category') # fix x axis to make the years look nice
fig.show()

*hover over each bar segment for more information*

In [6]:
saleval_2019 = df.drop(df[df.year != 2019].index)
saleval_2019 = pd.pivot_table(saleval_2019, index = ['date','customer_name'], values = ['sale_value','margin'], aggfunc = {'sale_value':'sum','margin':'sum'})
saleval_2019 = saleval_2019.reset_index(drop = False)
saleval_2019 = saleval_2019.sort_values(by=['customer_name'], ascending = True)

In [7]:
fig = px.bar(saleval_2019, x='date', y='sale_value', color = 'customer_name', title='2019 Daily Parts Sales by Customer')


fig.update_xaxes(rangeslider_visible=True)
fig.show()

*use the range finder to zoom in to a specific trading day*

In [8]:
saleval_2019_bymonth = df.copy()
saleval_2019_bymonth = saleval_2019_bymonth.drop(saleval_2019_bymonth[saleval_2019_bymonth.year != 2019].index)
saleval_2019_bymonth = pd.pivot_table(saleval_2019_bymonth, index = ['date','month'], values = ['sale_value','margin'], aggfunc = {'sale_value':'sum','margin':'sum'})
saleval_2019_bymonth = saleval_2019_bymonth.reset_index(drop = False)

In [9]:
fig = px.box(saleval_2019_bymonth, x = 'month', y = 'sale_value', hover_name = 'date', title='2019 Parts Sales Daily Average by Month')
fig.show()

*hover over the outliers to discover which date they represent*

In [10]:
customers = pd.pivot_table(df, index = ['customer_name'], values = ['inv_number','part_number','sale_value','margin','pricing_structure'], aggfunc = {'inv_number':'count','part_number':'count','sale_value':'sum','margin':'sum','pricing_structure':'first'})
customers = customers.rename(columns = {'inv_number':'total_invoices', 'part_number':'transaction_lines'})
customers = customers[['total_invoices','transaction_lines','sale_value','margin','pricing_structure']]
customers = customers.reset_index(drop=False)

In [11]:
fig = px.scatter(customers, x='transaction_lines', y='margin', log_x=False, log_y = False, size_max=100, hover_name = 'customer_name', color='pricing_structure', title='2017 - 2019 Revenue vs Transaction Lines per Customer')
fig.show()

*hover over each dot for more information*